In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

C:\Users\anilo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
SeaTemp='SW20'
WindSpeed='WC0'
Load='FAU'
df=df[df['SeaTemp']==SeaTemp]
df=df[df['WindSpeed']==WindSpeed]
df=df[df['Load']==Load]
result=df
result.dropna(inplace=True)
result.reset_index(drop=True,inplace=True)
abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
df_sample=result[abr_sensor_list]
x_train = df_sample.drop('Z02013', axis = 1) # Features
y_train = df_sample['Z02013']  # Target

In [3]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft=dft[dft['SeaTemp']==SeaTemp]
dft=dft[dft['WindSpeed']==WindSpeed]
dft=dft[dft['Load']==Load]
dft.dropna(inplace=True)
dft.reset_index(drop=True,inplace=True)
abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
dft=dft[abr_sensor_list]
x_test = dft.drop('Z02013', axis = 1) # Features
y_test = dft['Z02013']  # Target

In [4]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)

X_train_scaled = scaler.transform(x_train)
X_test_scaled = scaler.transform(x_test)

# LeakyReLU MSE

In [44]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_40 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_25 (Dropout)        (None, 320)               0         
                                                                 
 dense_51 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 128)               0         
                                                     

In [45]:
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 9s 5ms/step - loss: 70.5128 - mse: 70.5128 - val_loss: 0.0150 - val_mse: 0.0150
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.0497 - mse: 0.0497 - val_loss: 0.0142 - val_mse: 0.0142
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.0180 - mse: 0.0180 - val_loss: 9.3723e-04 - val_mse: 9.3723e-04
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.0012 - val_mse: 0.0012
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 9.9882e-04 - val_mse: 9.9882e-04
Epoch 6/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0011 - val_mse: 0.0011
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 8.6196e-04 - val_mse: 8.6196e-04
Epoch 8/100
1109/1109 [======

In [7]:
mse_neural, mae_neural = model.evaluate(X_test_scaled, y_test)
print('Mean squared error from neural net: ', (mse_neural))
print('Root Mean squared error from neural net: ', np.sqrt(mse_neural))

675/675 [==============================] - 1s 2ms/step - loss: 2.7158e-04 - mse: 2.7158e-04
Mean squared error from neural net:  0.00027157535078004
Root Mean squared error from neural net:  0.01647954340326333


#  LeakyReLU MAE

In [8]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 320)               11520     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 320)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 320)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               41088     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128)               0         
                                                      

In [9]:
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 70.5769 - mae: 4.9703 - val_loss: 0.0382 - val_mae: 0.1424
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0686 - mae: 0.1946 - val_loss: 0.0119 - val_mae: 0.0825
Epoch 3/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0263 - mae: 0.1202 - val_loss: 0.0068 - val_mae: 0.0637
Epoch 4/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0127 - mae: 0.0833 - val_loss: 0.0019 - val_mae: 0.0300
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0078 - mae: 0.0649 - val_loss: 0.0053 - val_mae: 0.0593
Epoch 6/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0064 - mae: 0.0587 - val_loss: 0.0037 - val_mae: 0.0457
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.0057 - mae: 0.0550 - val_loss: 0.0011 - val_mae: 0.0243
Epoch 8/100
1109/1109 [==============================]

In [10]:
mse_neural, mae_neural = model.evaluate(X_test_scaled, y_test)
print('Mean squared error from neural net: ', (mse_neural))
print('Root Mean squared error from neural net: ', np.sqrt(mse_neural))
print('Mean absolute error from neural net: ', mae_neural)

675/675 [==============================] - 1s 2ms/step - loss: 3.5429e-04 - mae: 0.0113
Mean squared error from neural net:  0.00035428759292699397
Root Mean squared error from neural net:  0.01882252886641415
Mean absolute error from neural net:  0.011305222287774086


# LeakyReLU MSE with Scale Y

In [11]:
scaler=StandardScaler()
scaler.fit(y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(y_train.values.reshape(-1, 1))

In [12]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 320)               11520     
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 320)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 320)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               41088     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 128)               0         
                                                      

In [13]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4892 - mse: 0.4892 - val_loss: 0.2632 - val_mse: 0.2632
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2710 - mse: 0.2710 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 3/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2180 - mse: 0.2180 - val_loss: 0.0905 - val_mse: 0.0905
Epoch 4/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1878 - mse: 0.1878 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1675 - mse: 0.1675 - val_loss: 0.0857 - val_mse: 0.0857
Epoch 6/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1583 - mse: 0.1583 - val_loss: 0.0698 - val_mse: 0.0698
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1554 - mse: 0.1554 - val_loss: 0.0750 - val_mse: 0.0750
Epoch 8/100
1109/1109 [==============================] 

In [14]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 1ms/step


In [15]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008674857446265012


#  LeakyReLU MAE with Scale Y

In [16]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 320)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 320)               0         
                                                                 
 dense_13 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 128)               0         
                                                      

In [17]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.5085 - mse: 0.5085 - val_loss: 0.1908 - val_mse: 0.1908
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2848 - mse: 0.2848 - val_loss: 0.1643 - val_mse: 0.1643
Epoch 3/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2204 - mse: 0.2204 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 4/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1931 - mse: 0.1931 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1708 - mse: 0.1708 - val_loss: 0.0864 - val_mse: 0.0864
Epoch 6/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1578 - mse: 0.1578 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1513 - mse: 0.1513 - val_loss: 0.0625 - val_mse: 0.0625
Epoch 8/100
1109/1109 [==============================] 

In [18]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 1ms/step


In [19]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008080137135249856


# ReLU MSE with Scale Y and 1 more layer

In [20]:
from tensorflow.keras.layers import Activation

model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(1))

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
lr_schedule = ExponentialDecay(initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 320)               11520     
                                                                 
 activation (Activation)     (None, 320)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_8 (Dropout)         (None, 320)               0         
                                                                 
 dense_17 (Dense)            (None, 256)               82176     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                      

In [21]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 9s 6ms/step - loss: 0.4142 - mse: 0.4142 - val_loss: 0.2638 - val_mse: 0.2638
Epoch 2/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.3147 - mse: 0.3147 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 3/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.3139 - mse: 0.3139 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 4/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.2724 - mse: 0.2724 - val_loss: 0.3644 - val_mse: 0.3644
Epoch 5/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.2328 - mse: 0.2328 - val_loss: 0.2264 - val_mse: 0.2264
Epoch 6/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1814 - mse: 0.1814 - val_loss: 0.2971 - val_mse: 0.2971
Epoch 7/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1826 - mse: 0.1826 - val_loss: 0.5304 - val_mse: 0.5304
Epoch 8/100
1109/1109 [==============================] 

In [22]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [23]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.06608865287750725


#  ELU MSE with Scale Y 

In [24]:
from tensorflow.keras.layers import ELU
from tensorflow.keras.regularizers import l1_l2

model = Sequential()

model.add(Dense(256, input_dim=X_train_scaled.shape[1], kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(32, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.005,
                               decay_steps=5000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 256)               9216      
                                                                 
 elu (ELU)                   (None, 256)               0         
                                                                 
 batch_normalization_17 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 128)               32896     
                                                                 
 elu_1 (ELU)                 (None, 128)               0         
                                                      

In [25]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 8s 5ms/step - loss: 4.5727 - mse: 0.5530 - val_loss: 2.7534 - val_mse: 0.4924
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 2.3783 - mse: 0.5339 - val_loss: 1.8107 - val_mse: 0.6650
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 1.7771 - mse: 0.5059 - val_loss: 1.4830 - val_mse: 0.4622
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 1.5657 - mse: 0.5121 - val_loss: 1.1492 - val_mse: 0.5275
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.5221 - mse: 0.4989 - val_loss: 1.3257 - val_mse: 0.4894
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 1.5107 - mse: 0.5083 - val_loss: 1.0509 - val_mse: 0.5793
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 1.3380 - mse: 0.4880 - val_loss: 0.7890 - val_mse: 0.3667
Epoch 8/100
1109/1109 [==============================] 

1109/1109 [==============================] - 5s 5ms/step - loss: 0.6279 - mse: 0.4476 - val_loss: 0.4861 - val_mse: 0.3438
Epoch 62/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.6112 - mse: 0.4356 - val_loss: 0.3553 - val_mse: 0.2176
Epoch 63/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.6324 - mse: 0.4448 - val_loss: 0.4987 - val_mse: 0.3229
Epoch 64/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.5893 - mse: 0.4366 - val_loss: 0.3754 - val_mse: 0.2257
Epoch 65/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5899 - mse: 0.4314 - val_loss: 0.4390 - val_mse: 0.2676
Epoch 66/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.6279 - mse: 0.4432 - val_loss: 0.4373 - val_mse: 0.3024
Epoch 67/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5924 - mse: 0.4357 - val_loss: 0.3649 - val_mse: 0.2132
Epoch 68/100
1109/1109 [==============================] - 5s 

In [26]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [27]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.018043350932021286


#  ELU MAE with Scale Y # 

In [28]:
from tensorflow.keras.layers import ELU
from tensorflow.keras.regularizers import l1_l2

model = Sequential()

model.add(Dense(256, input_dim=X_train_scaled.shape[1], kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(32, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.005,
                               decay_steps=5000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 256)               9216      
                                                                 
 elu_4 (ELU)                 (None, 256)               0         
                                                                 
 batch_normalization_21 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_14 (Dropout)        (None, 256)               0         
                                                                 
 dense_28 (Dense)            (None, 128)               32896     
                                                                 
 elu_5 (ELU)                 (None, 128)               0         
                                                      

In [29]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 4.7862 - mae: 0.4204 - val_loss: 1.9253 - val_mae: 0.5258
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 2.3218 - mae: 0.4070 - val_loss: 1.6068 - val_mae: 0.3590
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 2.2391 - mae: 0.3955 - val_loss: 2.7130 - val_mae: 0.3965
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.8923 - mae: 0.3967 - val_loss: 1.3716 - val_mae: 0.4739
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.4217 - mae: 0.3926 - val_loss: 6.6150 - val_mae: 0.8943
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.5467 - mae: 0.3921 - val_loss: 1.2491 - val_mae: 0.4158
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.3097 - mae: 0.3899 - val_loss: 0.9813 - val_mae: 0.3582
Epoch 8/100
1109/1109 [==============================] 

1109/1109 [==============================] - 5s 4ms/step - loss: 0.6049 - mae: 0.3751 - val_loss: 0.4509 - val_mae: 0.3191
Epoch 62/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.6282 - mae: 0.3746 - val_loss: 0.3986 - val_mae: 0.2781
Epoch 63/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5944 - mae: 0.3759 - val_loss: 0.4083 - val_mae: 0.2995
Epoch 64/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5905 - mae: 0.3713 - val_loss: 0.4565 - val_mae: 0.3227
Epoch 65/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5993 - mae: 0.3721 - val_loss: 0.3746 - val_mae: 0.2792
Epoch 66/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5747 - mae: 0.3719 - val_loss: 0.3969 - val_mae: 0.2859
Epoch 67/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5708 - mae: 0.3711 - val_loss: 0.5186 - val_mae: 0.3703
Epoch 68/100
1109/1109 [==============================] - 5s 

In [30]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [31]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.016635443127929393


In [32]:
from tensorflow.keras.layers import ELU, PReLU

model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1]))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(PReLU())
model.add(BatchNormalization())

model.add(Dense(32))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(1))

early_stopping = EarlyStopping(monitor='val_loss', patience=15)

lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 512)               18432     
                                                                 
 elu_8 (ELU)                 (None, 512)               0         
                                                                 
 batch_normalization_25 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_16 (Dropout)        (None, 512)               0         
                                                                 
 dense_33 (Dense)            (None, 256)               131328    
                                                                 
 p_re_lu (PReLU)             (None, 256)               256       
                                                      

In [33]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 11s 7ms/step - loss: 0.4799 - mse: 0.4799 - val_loss: 0.2088 - val_mse: 0.2088
Epoch 2/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2867 - mse: 0.2867 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 3/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2169 - mse: 0.2169 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 4/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.2022 - mse: 0.2022 - val_loss: 0.0766 - val_mse: 0.0766
Epoch 5/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1756 - mse: 0.1756 - val_loss: 0.0885 - val_mse: 0.0885
Epoch 6/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1585 - mse: 0.1585 - val_loss: 0.1144 - val_mse: 0.1144
Epoch 7/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 8/100
1109/1109 [==============================]

In [34]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [35]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.011678993052956093


In [36]:
from tensorflow.keras.layers import ELU, PReLU

model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

early_stopping = EarlyStopping(monitor='val_loss', patience=15)

lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 512)               18432     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_30 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 dense_39 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 256)               0         
                                                      

In [37]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 10s 7ms/step - loss: 0.4539 - mse: 0.4539 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 2/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2783 - mse: 0.2783 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 3/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2101 - mse: 0.2101 - val_loss: 0.0687 - val_mse: 0.0687
Epoch 4/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1833 - mse: 0.1833 - val_loss: 0.1005 - val_mse: 0.1005
Epoch 5/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1744 - mse: 0.1744 - val_loss: 0.0747 - val_mse: 0.0747
Epoch 6/100
1109/1109 [==============================] - 7s 7ms/step - loss: 0.1633 - mse: 0.1633 - val_loss: 0.0684 - val_mse: 0.0684
Epoch 7/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1480 - mse: 0.1480 - val_loss: 0.0585 - val_mse: 0.0585
Epoch 8/100
1109/1109 [==============================]

In [38]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [39]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007657092875344369


In [40]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.regularizers import l2

model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.01)))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, kernel_regularizer=l2(0.01)))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_regularizer=l2(0.01)))
model.add(PReLU())
model.add(BatchNormalization())

model.add(Dense(32, kernel_regularizer=l2(0.01)))
model.add(PReLU())
model.add(BatchNormalization())

model.add(Dense(1))

early_stopping = EarlyStopping(monitor='val_loss', patience=20)

lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 512)               18432     
                                                                 
 p_re_lu_2 (PReLU)           (None, 512)               512       
                                                                 
 batch_normalization_35 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_22 (Dropout)        (None, 512)               0         
                                                                 
 dense_45 (Dense)            (None, 256)               131328    
                                                                 
 p_re_lu_3 (PReLU)           (None, 256)               256       
                                                      

In [41]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 14s 9ms/step - loss: 5.1633 - mse: 0.5849 - val_loss: 2.5149 - val_mse: 0.3011
Epoch 2/100
1109/1109 [==============================] - 9s 8ms/step - loss: 1.5507 - mse: 0.3615 - val_loss: 0.7412 - val_mse: 0.1443
Epoch 3/100
1109/1109 [==============================] - 10s 9ms/step - loss: 0.7857 - mse: 0.3297 - val_loss: 0.5354 - val_mse: 0.1527
Epoch 4/100
1109/1109 [==============================] - 9s 8ms/step - loss: 0.6598 - mse: 0.3195 - val_loss: 0.5954 - val_mse: 0.2912
Epoch 5/100
1109/1109 [==============================] - 9s 8ms/step - loss: 0.6478 - mse: 0.3044 - val_loss: 0.6464 - val_mse: 0.3115
Epoch 6/100
1109/1109 [==============================] - 9s 8ms/step - loss: 0.6439 - mse: 0.3022 - val_loss: 0.4318 - val_mse: 0.1358
Epoch 7/100
1109/1109 [==============================] - 9s 8ms/step - loss: 0.6511 - mse: 0.3066 - val_loss: 0.5151 - val_mse: 0.1962
Epoch 8/100
1109/1109 [==============================

In [42]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [43]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.04051371892087501
